In [1]:
import requests
import time
from tempfile import TemporaryDirectory
from IPython.display import clear_output, FileLink, Image, display
from zipfile import ZipFile
from pyarrow import parquet
import json
import os

COSMOS_BASE_URL = "http://cosmos0001.chtc.wisc.edu:8088/cosmos_service"
SAMPLE_PDF_URL = "https://www.nature.com/articles/s41591-020-0883-7.pdf"
SAMPLE_PDF_PATH = "sidarthe.pdf"


### Submit a PDF for Processing

Make a POST request to the base URL containing a multipart form with the contents of the file in the `pdf` field.
Other form fields include:
- `compress_images` (default `True`): Return compressed JPG images if `True`, uncompressed PNGs otherwise.  

In [ ]:
# Download a local copy of the PDF

with open(SAMPLE_PDF_PATH, 'wb') as pdf_writer:
    pdf_writer.write(requests.get(SAMPLE_PDF_URL).content)


In [2]:
# Submit the locally copied PDF to the COSMOS processing pipeline

submit_endpoint = COSMOS_BASE_URL + '/process/'

with open(SAMPLE_PDF_PATH, 'rb') as pdf_to_parse:
    file_form = {'pdf': pdf_to_parse }
    data_form = {'compress_images': False }

    response = requests.post(submit_endpoint, files=file_form, data=data_form)

    response_data = response.json()
    job_id = response_data['job_id']

    status_endpoint = response_data['status_endpoint']
    results_endpoint = response_data['result_endpoint']

    print(f"Message: {response_data['message']}\n"
          f"Job ID: {response_data['job_id']}\n"
          f"Status Endpoint: {status_endpoint}\n"
          f"Results Endpoint: {results_endpoint}")


SSLError: HTTPSConnectionPool(host='cosmos0001.chtc.wisc.edu', port=8088): Max retries exceeded with url: /cosmos_service/process/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1007)')))

### Monitor COSMOS Processing Status

Poll the URL suffix returned in the `status_endpoint` field of the initial response object.
For the sample document, processing can take up to 5 minutes. The `/status` endpoint
will return several fields, including a `job_completed` field and an `error` field which initially 
`false` and `null` respectively. If the `job_completed` field becomes `true`, then the job completed 
successfully, and if the `error` field becomes non-`null` then the job failed. 

In [ ]:
POLL_COUNT = 80
POLL_INTERVAL = 5

job_done = False

for i in range(POLL_COUNT):
    response = requests.get(status_endpoint)
    response_data = response.json()
    clear_output(wait=True)
    print(f"Polled status endpoint {i} times:\n{response_data}")
    job_done = response_data['error'] or response_data['job_completed']
    if job_done:
        break
    time.sleep(POLL_INTERVAL)

if not job_done:
    print(f"ERROR: Job not complete after {POLL_COUNT * POLL_INTERVAL} seconds.")
elif response_data['error']:
    print(f"An unexpected error occurred: {response_data['error']}")
else:
    print(f"Job succeeded after {response_data['time_processing']} seconds.\n"
          f"Results can be viewed at {results_endpoint}")


### Obtain Results

Once the `/status` endpoint indicates that `job_completed` is `true`, the following endpoints will be available via GET request:
- `/result/text`: The text extracted from the document, alongside its bounding boxes and classification, as JSON.
- `/result/extractions/{extraction_type}`: The bounding boxes of individual extraction types, and temporary links to their images, as JSON. 
  Valid extraction types are `equations`, `figures`, and `tables`.
- `/result/images/{image_id}`: Temporary image hosting for the cosmos extractions.
- `/result`: The entire contents of the cosmos processing job as a zip file, containing images of extracted pdf regions and parquet files with those extractions' metadata.

All results are available for 24 hours after image processing completes, after which they will be deleted.

### Extractions (JSON)

In [ ]:
# Extracted document text and bounding boxes
text_data = requests.get(f"{results_endpoint}/text")
print(json.dumps(text_data.json(), indent=2))


In [ ]:
# Extracted document equations, bounding boxes, and images
equation_data = requests.get(f"{results_endpoint}/extractions/equations")
print(json.dumps(equation_data.json(), indent=2))


In [ ]:
# Extracted document figures, bounding boxes, and images
figure_data = requests.get(f"{results_endpoint}/extractions/figures")
print(json.dumps(figure_data.json(), indent=2))


In [ ]:
# Extracted document tables, bounding boxes, and images
table_data = requests.get(f"{results_endpoint}/extractions/tables")
print(json.dumps(table_data.json(), indent=2))


### Full Results (Zip)

In [ ]:
ZIP_DOWNLOAD_PATH = SAMPLE_PDF_PATH.replace('.pdf', '.zip')
with open(ZIP_DOWNLOAD_PATH, 'wb') as writer:
    writer.write(requests.get(results_endpoint).content)

print(f'You can access the downloaded zip file at {FileLink(ZIP_DOWNLOAD_PATH)} .\nIt contains the following files:')

with ZipFile(ZIP_DOWNLOAD_PATH, 'r') as zipfile:
    zipfile.printdir()


### View Figures

The zip file contains a number of image files, as well as apache parquet files containing information about those images.
Each parquet file contains information about a separate extraction type (tables, figures, and equations). The names of each
parquet file are based on the name of the input PDF file.

In [ ]:
def show_extractions(parquet_path, bb_column, page_column):
    """Utility function to show every extraction from a single COSMOS Parquet file"""
    with (TemporaryDirectory() as td, ZipFile(ZIP_DOWNLOAD_PATH, 'r') as zipfile):
        # extract the parquet file from the zip archive
        zipfile.extract(parquet_path, td)
        # read the parquet file
        parquet_table = parquet.read_table(os.path.join(td, parquet_path))
        pandas_data: pd.DataFrame = parquet_table.to_pandas()
        # TODO the img_path column currently contains the absolute path of the image on the server,
        # rather than the relative path from the root of the zip folder
        pandas_data['img_pth'] = pandas_data['img_pth'].replace('/tmp/tmp.*/','',regex=True)

        # Print each extraction alongside its score, page, and bounding box
        for idx, row in pandas_data.iterrows():
            img_path = row['img_pth']
            zipfile.extract(img_path, td)
            print(f"Extracted Item #{idx+1}\n"
                  f"Page: {row[page_column]}\n"
                  f"Bounding Box: {row[bb_column]}\n"
                  f"Score: {row['postprocess_score']}")
            display(Image(os.path.join(td,img_path)))


In [ ]:
# Figures
show_extractions(
    'paper.pdf'.replace('.pdf','_figures.parquet'), 
    'obj_bbs',
    'obj_page')


In [ ]:
# Equations
show_extractions(
    SAMPLE_PDF_PATH.replace('.pdf','_equations.parquet'), 
    'equation_bb',
    'equation_page')


In [ ]:
# Tables
show_extractions(
    SAMPLE_PDF_PATH.replace('.pdf','_tables.parquet'), 
    'obj_bbs',
    'obj_page')


# Validate Results

COSMOS provides the /healthcheck/evaluate endpoint to evaluate the output of a COSMOS job against an expected set of region bounding boxes. Expected bounding boxes can be generated
by the [COSMOS tagger tool](https://github.com/uw-cosmos/cosmos-visualizer), but should be converted from XML to JSON for use with the health check endpoint. The health check endpoint
returns the [mean average precision score](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision) for each of the 3 relevant region
identification classes (figures, tables, and equations) according to the following procedure:

* For each threshold between 50% and 95%, in 5% intervals:
* Label each Cosmos extraction as a true positive if its intersection-over-union with a ground truth region is greater than the threshold
* Compute the [average precision](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Average_precision) of the set of extractions at the given threshold
  * Average Precision is a measure of the change in precision and recall of the model as results with lower confidence scores are considered
* Average the average precision score from each i-o-u threshold

In [ ]:
evaluate_endpoint = COSMOS_BASE_URL + f'/healthcheck/evaluate/{job_id}'
with open(SAMPLE_PDF_PATH + '.json', 'rb') as expected_regions:

    response = requests.post(evaluate_endpoint, json=json.loads(expected_regions.read()))

    print(json.dumps(response.json(),indent=2))
